In [1]:
!nvidia-smi

Sat Sep 24 19:29:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 32%   45C    P2    61W / 170W |   4310MiB / 12045MiB |     54%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# 安装必要依赖
!pip install -r requirements.txt
!sudo apt-get install espeak -y

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 KB 3.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 3.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 3.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.7/25.7 MB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.3/238.3 KB 4.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 2.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 KB 3.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 4.3 MB

In [4]:
# 音频解码
!sudo apt-get install -y sox

Reading package lists... Done
Building dependency tree       
Reading state information... Done
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 82 not upgraded.


In [7]:
%cd /hy-tmp/vits/voice

/hy-tmp/vits/voice


In [8]:
%%bash
for x in ./*.wav
do 
  b=${x##*/}
  sox -v 0.90 $b -r 22050 tmp_$b
  rm -rf $b
  mv tmp_$b $b
done

In [7]:
# 再次安装依赖
%cd /root/vits-mandarin-biaobei/monotonic_align
!python setup.py build_ext --inplace
%cd ..

/root/vits-mandarin-biaobei/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.8/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /root/vits-mandarin-biaobei/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'monotonic_align.core' extension
creating build
creating build/temp.linux-x86_64-3.8
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.8/dist-packages/numpy/core/include -I/usr/include/python3.8 -c core.c -o build/temp.linux-x86_64-3.8/core.o
creating /root/vits-mandarin-biaobei/monotonic_align/monotonic_align
x86_64-lin

In [4]:
# 数据处理
%cd /hy-tmp/vits
!python preprocess.py --text_index 1 --text_cleaners chinese_cleaners1 --filelists filelists/paimen_train.txt filelists/paimen_test.txt

/hy-tmp/vits
START: filelists/paimen_train.txt
START: filelists/paimen_test.txt


In [ ]:
%cd /hy-tmp/vits
!python train.py -c configs/biaobei_base.json -m biaobei_base

/hy-tmp/vits
[INFO] {'train': {'log_interval': 200, 'eval_interval': 1000, 'seed': 1234, 'epochs': 2000, 'learning_rate': 0.0002, 'betas': [0.8, 0.99], 'eps': 1e-09, 'batch_size': 16, 'fp16_run': True, 'lr_decay': 0.999875, 'segment_size': 8192, 'init_lr_ratio': 1, 'warmup_epochs': 0, 'c_mel': 45, 'c_kl': 1.0}, 'data': {'training_files': 'filelists/paimen_train.txt.cleaned3', 'validation_files': 'filelists/paimen_test.txt.cleaned', 'text_cleaners': ['chinese_cleaners1'], 'max_wav_value': 32768.0, 'sampling_rate': 22050, 'filter_length': 1024, 'hop_length': 256, 'win_length': 1024, 'n_mel_channels': 80, 'mel_fmin': 0.0, 'mel_fmax': None, 'add_blank': True, 'n_speakers': 0, 'cleaned_text': True}, 'model': {'inter_channels': 192, 'hidden_channels': 192, 'filter_channels': 768, 'n_heads': 2, 'n_layers': 6, 'kernel_size': 3, 'p_dropout': 0.1, 'resblock': '1', 'resblock_kernel_sizes': [3, 7, 11], 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]], 'upsample_rates': [8, 8, 2, 2], 'u

In [29]:
# 处理有问题的数据集
from utils import load_wav_to_torch, load_filepaths_and_text
from text import text_to_sequence, cleaned_text_to_sequence
import os
import commons
import torch
from mel_processing import spectrogram_torch
sampling_rate = 22050
max_wav_value = 32768.0
filter_length = 1024
hop_length = 256
win_length = 1024
cleaned_text=True
text_cleaners=["chinese_cleaners1"]
add_blank=True

def get_audio(filename):
    audio, sampling_rate = load_wav_to_torch(filename)
    if sampling_rate != sampling_rate:
        raise ValueError("{} {} SR doesn't match target {} SR".format(
            sampling_rate, sampling_rate))
    audio_norm = audio / max_wav_value
    audio_norm = audio_norm.unsqueeze(0)
    spec_filename = filename.replace(".wav", ".spec.pt")
    print(spec_filename)
    if os.path.exists(spec_filename):
        spec = torch.load(spec_filename)
    else:
        spec = spectrogram_torch(audio_norm, filter_length,
                                 sampling_rate, hop_length, win_length,
                                 center=False)
        spec = torch.squeeze(spec, 0)
        torch.save(spec, spec_filename)
    return spec, audio_norm


def get_text(text):
    if cleaned_text:
        text_norm = cleaned_text_to_sequence(text)
    else:
        text_norm = text_to_sequence(text, text_cleaners)
    if add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [30]:
with open("filelists/paimen_train.txt.cleaned2") as f:
    with open("filelists/paimen_train.txt.cleaned3","w") as f2:
        for file in f.read().split("\n"):
            name = file.split("|")
            if len(name) == 2:
                try:
                    # get_audio(name[0])
                    get_text(name[1])
                    f2.write("{}\n".format(file))
                except Exception as e:
                    print(e)
                    print(file)
                        

'9'
voice/0217.wav|wo3 kan4 ni3 gang1 cai2 shi4 99 fen1 dou1 shi4 yong3 qi4 le ba 。
'7'
voice/0354.wav|ru2 guo3 na4 yi1 zhao1 zhen1 de wu2 di2 de hua4 ，700 lei2 dian4 jiang1 jun1 ye3 jiu4 wu2 cong2 tan2 qi3 ba 。
'6'
voice/1011.wav|ru2 guo3 yao4 he1 ba1 li2 mao4 xian3 tuan2 he2 zuo4 tan4 suo3 mi4 qie4 jiu4 yi2 ding4 yao4 zuo4 hao3 zhun3 bei4 。 mi4 jing1 shi4 hen3 wei1 xian3 de ， chu1 fa1 qian2 yao4 duo1 jian3 cha2 ji3 bian4 wu4 zi1 he2 gong1 ju4 ， bu4 neng2 da4 yi4 。 wo3 ting1 shuo1 zai4 zhe4 xie1 mi4 jing4 li3 guang1 shi4 mao4 xian3 jia1 xie2 hui4 tong3 ji4 guo4 de si3 fa3 jiu4 you3 463 zhong3 。 ru2 guo3 ba3 ni3 tou2 bu4 xiao3 xin1 hai4 si3 le ni3 ， wo3 he2 ta1 de liang2 xin1 dou1 hui4 hen3 bu4 an1 de 。 wei4 shen2 me si3 de zhi3 you3 wo3 。 n2 ， gen1 ju4 tong3 ji4 ， yin1 wei4 wo3 hui4 fei1 ， suo3 yi3 ke3 yi3 zhi4 shao3 mian3 yi4 qi2 zhong1 de 322 zhong3 si3 fa3 。 wo3 hu1 ran2 bu4 tai4 xiang3 qu4 mi4 jing4 tan4 xian3 le 。
'7'
voice/2034.wav|ni3 zhi1 dao4 ma ？ na4 wei4 shan1 hu2 gong1 xin